In [4]:
import pandas as pd
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn as sk
import requests
import time
from sklearn.model_selection import train_test_split
from sklearn import linear_model 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler

In [5]:
#데이터 가져오기
df = pd.read_csv("raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("raw_data/cgoongoo1.csv", encoding ='utf-8')

In [6]:
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district'], axis=1, inplace =True)

In [7]:
df_merged.columns

Index(['year', 'code', 'sales_total', 's_store_no_of_store',
       's_store_no_of_opening', 's_store_no_of_closing', 's_work_female_40s',
       's_work_female_50s', 's_work_female_60s', 'b_facil_total',
       's_float_male', 's_float_female', 'b_apt_avg_price',
       'b_income_avg_monthly_inc', 's_work_female', 's_work_male',
       'sales_weekday', 'sales_female', 'sales_2030s', 'sales_06_11',
       'sales_11_14', 'sales_14_17', 'sales_17_21', 'sales_21_24', 'cgoongoo'],
      dtype='object')

In [8]:
df_cgoongu_gb=df_merged.groupby(['year','cgoongoo', 'code']).agg({
    "sales_total":"sum",
    "s_store_no_of_store":"sum", #이걸 mean으로 해야할지 고민이 필요
    "s_store_no_of_opening":"sum",
    "s_store_no_of_closing":"sum",
    "sales_weekday":"sum",
    "sales_female":"sum",
    "sales_2030s":"sum",
    "sales_06_11":"sum",
    "sales_11_14":"sum",
    "sales_14_17":"sum",
    "sales_17_21":"sum",
    "sales_21_24":"sum",
    "s_work_female_40s":"mean",
    "s_work_female_50s":"mean",
    "s_work_female_60s":"mean",
    "b_facil_total":"mean",
    "s_float_male":"mean",
    "s_float_female":"mean",
    'b_apt_avg_price':'mean',
    'b_income_avg_monthly_inc':'mean',
    's_work_female':'mean',
    's_work_male':'mean',
    'b_income_avg_monthly_inc':'mean'
    
})
df_cgoongu_gb.tail(3)

sales_total  s_store_no_of_store  \
year cgoongoo code                                          
2019 11740    CS300015  5.912349e+09                   49   
              CS300016  5.516874e+09                  496   
              CS300017  2.322398e+09                   95   

                        s_store_no_of_opening  s_store_no_of_closing  \
year cgoongoo code                                                     
2019 11740    CS300015                      1                      1   
              CS300016                      8                      7   
              CS300017                      0                      1   

                        sales_weekday  sales_female  sales_2030s  sales_06_11  \
year cgoongoo code                                                              
2019 11740    CS300015       4.290286      3.237088     0.762611     0.080720   
              CS300016      25.429959      9.793725     6.189211     7.338632   
              CS300017       5.827562      4.553235     3.578226     0.315342   

                        sales_11_14  sales_14_17  ...  s_work_female_40s  \
year cgoongoo code                                ...                      
2019 11740    CS300015     2.089863     2.577843  ...             64.250   
              CS300016     7.644050    10.612261  ...             50.375   
              CS300017     3.444519     3.303651  ...             51.000   

                        s_work_female_50s  s_work_female_60s  b_facil_total  \
year cgoongoo code                                                            
2019 11740    CS300015          64.083333          34.333333     112.500000   
              CS300016          51.750000          28.187500     108.156250   
              CS300017          52.722222          33.666667     118.888889   

                         s_float_male  s_float_female  b_apt_avg_price  \
year cgoongoo code                                                       
2019 11740    CS300015  194778.250000   189097.833333     2.004435e+08   
              CS300016  158058.328125   154031.750000     1.826516e+08   
              CS300017  167489.388889   165458.055556     1.682300e+08   

                        b_income_avg_monthly_inc  s_work_female  s_work_male  
year cgoongoo code                                                            
2019 11740    CS300015              3.384987e+06     253.500000   274.166667  
              CS300016              3.370583e+06     205.921875   221.187500  
              CS300017              3.212065e+06     216.277778   193.944444  

[3 rows x 22 columns]

In [9]:
# # df 저장
# #index에 names=['year', 'cgoongoo', 'code']가 들어있다
# df_cgoongu_gb.to_csv('raw_data/df_cgoongu_gb.csv', index=True, encoding='utf-8')

In [10]:
df_cgoongu_gb.reset_index(inplace=True)

In [12]:
df_cgoongu_gb.head()

,year,cgoongoo,code,sales_total,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,sales_weekday,sales_female,sales_2030s,...,s_work_female_40s,s_work_female_50s,s_work_female_60s,b_facil_total,s_float_male,s_float_female,b_apt_avg_price,b_income_avg_monthly_inc,s_work_female,s_work_male
0,2018,11110,CS100001,4.711364e+10,624,22,30,17.282523,7.734313,7.488426,...,58.333333,44.833333,17.708333,145.250000,70010.583333,70020.875000,2.194223e+08,3.878552e+06,239.875000,276.708333
1,2018,11110,CS100002,3.764873e+09,55,5,3,8.872807,3.788772,4.428286,...,66.166667,49.833333,18.916667,161.166667,79649.458333,76463.958333,2.364741e+08,3.766325e+06,271.416667,332.666667
2,2018,11110,CS100003,3.625603e+09,54,2,1,8.907585,4.313511,5.107181,...,62.909091,46.363636,19.181818,155.272727,88827.045455,89418.727273,2.027983e+08,3.760143e+06,273.818182,264.727273
3,2018,11110,CS100004,1.541111e+10,187,4,6,11.915524,6.677956,7.564242,...,68.058824,47.000000,16.470588,150.823529,65091.500000,65892.588235,2.618564e+08,4.032644e+06,273.235294,278.529412
4,2018,11110,CS100005,7.559278e+09,222,23,10,12.324600,7.842999,7.735840,...,53.235294,40.294118,17.764706,164.411765,79970.441176,79983.970588,2.035326e+08,3.719739e+06,228.000000,280.000000


In [23]:
scale_columns

['cgoongoo',
 'sales_total',
 's_store_no_of_store',
 's_store_no_of_opening',
 's_store_no_of_closing',
 'sales_weekday',
 'sales_female',
 'sales_2030s',
 'sales_06_11',
 'sales_11_14',
 'sales_14_17',
 'sales_17_21',
 'sales_21_24',
 's_work_female_40s',
 's_work_female_50s',
 's_work_female_60s',
 'b_facil_total',
 's_float_male',
 's_float_female',
 'b_apt_avg_price',
 'b_income_avg_monthly_inc',
 's_work_female',
 's_work_male']

In [24]:
#로버스트 스케일링
from sklearn.preprocessing import RobustScaler
scale_columns = [i for i in df_cgoongu_gb.columns if i not in ['year', 'cgoongoo', 'code',]]

rb = RobustScaler()
rb.fit(df_cgoongu_gb[scale_columns])
df_robust_scaled = rb.transform(df_cgoongu_gb[scale_columns])
df_robust_scaled = pd.DataFrame(df_robust_scaled, columns=scale_columns)
df_robust_scaled2= pd.concat([df_cgoongu_gb[['cgoongoo','code']],df_robust_scaled], axis=1)
df_robust_scaled.columns
model= sm.OLS.from_formula("sales_total ~ C(cgoongoo) +C(code) +"+" + ".join(scale_columns[2:]), data=df_robust_scaled2)
result =  model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            sales_total   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.829
Method:                 Least Squares   F-statistic:                     123.1
Date:                Thu, 26 Dec 2019   Prob (F-statistic):               0.00
Time:                        17:12:19   Log-Likelihood:                -2508.2
No. Observations:                2220   AIC:                             5194.
Df Residuals:                    2131   BIC:                             5702.
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               